In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping

c:\Users\acer\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\acer\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\acer\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/

In [2]:
# 1. Load data
train_df = pd.read_csv('sign_mnist_train.csv')
test_df = pd.read_csv('sign_mnist_test.csv')

In [3]:
# 2. Pisahkan fitur dan label
X_train = train_df.drop('label', axis=1).values / 255.0
y_train = train_df['label'].values
X_test = test_df.drop('label', axis=1).values / 255.0
y_test = test_df['label'].values

In [4]:
# 3. Reshape gambar menjadi (28, 28, 1)
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In [5]:
# 4. One-hot encoding label
y_train_cat = to_categorical(y_train, num_classes=25)
y_test_cat = to_categorical(y_test, num_classes=25)


In [6]:
# 5. Split train/val
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train_cat, test_size=0.1, random_state=42
)

In [7]:
# 6. Bangun model CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(25, activation='softmax')
])

c:\Users\acer\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
# 7. Training model
early_stop = EarlyStopping(patience=3, restore_best_weights=True)
history = model.fit(
    X_train_split, y_train_split,
    validation_data=(X_val_split, y_val_split),
    epochs=15,
    batch_size=64,
    callbacks=[early_stop]
)

Epoch 1/15
387/387 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.4837 - loss: 1.6808 - val_accuracy: 0.8827 - val_loss: 0.4977
Epoch 2/15
387/387 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.8433 - loss: 0.4670 - val_accuracy: 0.9632 - val_loss: 0.1526
Epoch 3/15
387/387 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.9238 - loss: 0.2286 - val_accuracy: 0.9858 - val_loss: 0.0686
Epoch 4/15
387/387 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.9555 - loss: 0.1405 - val_accuracy: 0.9931 - val_loss: 0.0501
Epoch 5/15
387/387 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.9681 - loss: 0.1007 - val_accuracy: 0.9985 - val_loss: 0.0113
Epoch 6/15
387/387 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.9781 - loss: 0.0717 - val_accuracy: 0.9989 - val_loss: 0.0101
Epoch 7/15
387/387 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.9798 - loss: 0.0624 - val_accuracy: 0.9996 - val_loss: 0.0049
Epoch 8/15
387/387 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.9845 - loss: 0.0473 - val_acc

In [10]:
# 8. Evaluasi
test_loss, test_acc = model.evaluate(X_test, y_test_cat)
print(f'\nAkurasi pada data uji: {test_acc:.4f}')

225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9225 - loss: 0.3041

Akurasi pada data uji: 0.9225


In [11]:
# 9. Simpan model
model.save("asl_sign_model.h5")